In [401]:
#Importing the required libraries.
import pandas as pd #for dataframes and reading csv files.
import numpy as np # for math operations(sqrt)
import matplotlib.pyplot as plt #for plotting the graphs



In [402]:
file_day_od="data/day_od.csv" #directory of the file(file_path)
#csv (comma seperated values, delimiter is comma)
data_day_od=pd.read_csv(file_day_od,sep=",",encoding='utf-16', header=0)
                       
#quoting=2,quotechar='"',escapechar='\r' )
#comment: explicit header=0 to replace column names;  quoting=2 "Non-Numerical"; es

#seperate the columns by name
headerName=["MONTH","DOW","ORIGIN","DESTINATION","CUST_CLASS","COD_COUNTRY","COD_PRO","PRO_COM","FLOW"]
data_day_od.columns=headerName
day_od=pd.DataFrame(data_day_od)#dataFrame of the file(can be read more easily than csv)

day_od.head()

#data_day_od

# Getting DataFrame for distinct_users_day.csv
file_distinct_users_day="data/distinct_users_day.csv"
data_distinct_users_day=pd.read_csv(file_distinct_users_day,sep=",",encoding="latin-1")
                                    #,quoting=3,escapechar="\"",encoding="latin-1")
distinct_users_day=pd.DataFrame(data_distinct_users_day)
#distinct_users_day

# Getting DataFrame for codici_istat_comune.csv
file_codici_istat_comune="data/codici_istat_comune.csv"
data_codici_istat_comune=pd.read_csv(file_codici_istat_comune,sep=",",delimiter=',',header=0,quoting=3,escapechar="\"",encoding="latin-1")
codici_istat_comune=pd.DataFrame(data_codici_istat_comune)
#codici_istat_comune

# Getting DataFrame for codici_istat_provincia.csv
file_codici_istat_provincia="data/codici_istat_provincia.csv"
data_codici_istat_provincia=pd.read_csv(file_codici_istat_provincia,sep=",",header=0,quoting=3,escapechar="\"",encoding="latin-1")
codici_istat_provincia=pd.DataFrame(data_codici_istat_provincia)
#codici_istat_provincia

# Getting DataFrame for codici_nazioni.csv
import csv
file_codici_nazioni="data/codici_nazioni.csv"
data_codici_nazioni=pd.read_csv(file_codici_nazioni,delimiter=',',header=0,quoting=2, 
                                error_bad_lines=True,encoding="utf-16",quotechar = '"',escapechar='\r')
codici_nazioni=pd.DataFrame(data_codici_nazioni)
 #codici_nazioni[108:112]

## POINT 4

 Study of the visitors' fluxes: let's assume to be asked to provide indications of how to invest resources to improve the mobility toward Padova. Defined the three main directions of visitors and commuter getting to Padova by the Italian highways (sud: A13 toward Bologna-Roma / west: A4 toward Milano-Torino / north-east: A4 toward Venice-Trieste), evaluate which of the three directions has to be prioritized:
 1. Consider a simplified case involving only the mid-range mobility, based on the number of visitors/commuters from the nearby regions only
 * Consider the provinces located on the three directions that are mostly contributing to the flow of weekend visitors and working daily commuters by performing a more detailed study of the fluxes based on the day of the week
 * _[OPEN]_ Use the data available to provide what you believe is the best possible answer 

Assuming that it's asked

The requirement for this part is to provide indications on how to invest resources to improve the mobility toward Padova, considering the the three main directions toward Padova, that are, considering the Italian highways:

-west: A4 toward Milano-Torino
-south: A13 toward Bologna-Roma 
-north-east: A4 toward Venice-Trieste. 



To simplyfy our study at first we consider only nearby regions, we've selected one region for each of the different directions: Lombardia at west, Emilia Romagna at south and Friuli Venezia Giulia at north-east.

The first step is to obtain all the data for the selected regions: we first need to find region's cod (info_region func and d\[reg\[i\]\] dataframe) in order to extract from the dataset data_od_od the information for our selected regions. (data_reg\[reg\[i\]\] dataset, contains all provinces)

We can then  calculate the total flow (flow_region function) 




In [403]:
#For the regions of interest insert the region capital names and the region names
cap=['Milano', 'Bologna', 'Trieste']
reg=['Lombardia', 'Emilia Romagna', 'Friuli Venezia Giulia']

#Function to obtain the info of the regions of interest 
def info_regions(codici_istat_provincia, capoluogo):
    cod_reg=codici_istat_provincia[codici_istat_provincia['PROVINCIA'] == capoluogo]['COD_REG'].values[0] 
    info_reg=codici_istat_provincia[codici_istat_provincia['COD_REG'] == cod_reg]
    return info_reg

#Create a dictionary of dataframes for the different regions (cod info only)
d = {} 
cod_int=[]
for c in cap:
    d[reg[cap.index(c)]] = pd.DataFrame(info_regions(codici_istat_provincia,c))


#dataframe d[key] with key=reg[i], data: |COD_REG|COD_PRO|PROVINCIA|PROV_SIGLA
#print(d)
#d[reg[0]]
#print(d[reg[i]].iloc[:,1])
#d[reg[0]].iat[1,1]

#DATA OF THE REGIONS OF INTEREST

#Returns a dataset with all the info (data_day_od) of the regions of interest
def data_regions(data_day_od, cod_all_prov):
    data_reg=data_day_od[data_day_od['COD_PRO'].isin(cod_all_prov)]
    return data_reg

#Define a dictionary with the dataframe for the different regions with all the info from data_day_od
#data_reg[reg[i]]  MONTH|DOW|ORIGIN|DESTINATION|CUST_CLASS|COD_CONTRY|COD_PRO|PRO_COM|FLOW
data_reg = {} 
for i in range(len(reg)):
    data_reg[reg[i]] = pd.DataFrame(data_regions(data_day_od,d[reg[i]]['COD_PRO'].values[:]))
    

In [404]:
#######4.1: STUDY OF THE FLUXES OF NEARBY REGIONS TO FIND WHICH OF THE THREE DIRECTIONS HAS TO BE PRIORITIZED######

#Function to calculate the total flow of the regions of interest 
def flow_regions(info_data, cod_all_prov):
    flow_sum=info_data.groupby(info_data['COD_PRO'].isin(cod_all_prov))['FLOW'].sum()
    flow_sum=flow_sum.reset_index()
    return flow_sum['FLOW'][flow_sum['COD_PRO'] == True][0]
    
#Create a dictionary for the the total flux for the different regions 
d_flow = {}
for i in range(len(reg)):
    d_flow[reg[i]] = flow_regions(data_reg[reg[i]], d[reg[i]]['COD_PRO'].values[:])
    
    
#Dataframe with the total flux for the regions of interest, flow: REGION|FLOW|
flow= pd.DataFrame(d_flow.items(), columns=['REGION', 'FLOW']).sort_values(by='FLOW', ascending=False)

#############################

#Display the results
import plotly.graph_objs as go
import plotly.figure_factory as ff

#text = [['Region', 'Flow'], [flow.iat[0,0],flow.iat[0,1]], [flow.iat[1,0],flow.iat[1,1]],[flow.iat[2,0],flow.iat[2,1]]]
#font=['#FCFCFC', '#254607', '#417a0c', '#5fb311']
table_color =  [[0, '#272D31'],[.5, '#ffffff'],[1, '#ffffff']]

# Initialize a figure with ff.create_table(table_data)
fig = ff.create_table(flow, height_constant=80, colorscale=table_color) # font_colors=font)#colorscale

color_dict = ['#254607','#417a0c','#5fb311']
fig.add_trace(go.Bar(x = flow["REGION"], y = flow["FLOW"],
                        marker_color=color_dict,
                        name='Flow',
                        xaxis='x2', yaxis='y2',
                        ))
    
fig.update_layout(
    title_text = 'Total flow for the nearby regions', #of the table
    margin = {'t':60, 'b':80, },
    xaxis = {'domain': [0, .4]},
    xaxis2 = {'domain': [0.5, 1.], 'title':'Region'},
    yaxis2 = {'domain': [0, 1], 'anchor': 'x2', 'title': 'Flow'}
)

fig.show()



As we can see from this plot the region with the highest flux is "Lombardia", followed by "Emilia Romagna" and "Friuli Venezia Giulia". 
Of the three directions, the Italian highways A4, torward Milano-Torino, should be proritized. 


## POINT 4.2

Consider the provinces located on the three directions that are mostly contributing to the flow of weekend visitors and working daily commuters by performing a more detailed study of the fluxes based on the day of the week <br>


Firstly we'd like to know whch of the provinces of the selected regions are contributing the most, we select the first three provinces for each region (flow_province function). We then study the fluxes based on the days of the week. 

In [405]:
########## 4.2: STUDY OF THE FLUXES OF THE MOSTLY CONTRIBUTING PROVINCES BASED ON THE DAY OF THE WEEK #########

#SELECTION OF THE PROVINCES THAT ARE MOSTLY CONTRIBUTING BASED ON THE FLOW 

#Functions that finds the mostly contributing provinces for each region:
#Returns the total flux for each province and the code of the selected provinces
def flow_province(data_regions):
    flow_province=data_regions.groupby('COD_PRO').FLOW.sum()
    flow_sorted=flow_province.sort_values(ascending=False)
    #flow_selected=flow_sorted.loc[flow_sorted[:] >= 1e4].reset_index()
    flow_selected=flow_sorted.head(n=3).reset_index()  #We choose the first 3 with the highest flux
    cod_pro_selected=flow_selected['COD_PRO'].values[:]
    return flow_selected, cod_pro_selected

d_flow_province = {} #for function output
cod_pro_selected={} #for function output
flow_pro={} #to built a dictionary with a flow dataframe for the different regions
pro_selected={}  #to built a dictionary with the code of the selected provinces

for i in range(len(reg)): 
    d_flow_province, cod_pro_selected = flow_province(data_reg[reg[i]]) #function returns two different outputs
    flow_pro[reg[i]]= d_flow_province
    pro_selected[reg[i]]=cod_pro_selected
    

#Associating each province code the province name
#flow of the province for a region flow_pro[reg[i]]: 'COD PRO', 'NAME', 'FLOW'

for i in range(len(reg)):
    name_l=[]
    for cod in flow_pro[reg[i]]['COD_PRO'].values[:]:
        if cod in d[reg[i]]['COD_PRO'].values[:]:
            where=np.argwhere(d[reg[i]]['COD_PRO'].values[:]==cod)
            idx=where[0][0] 
            name=d[reg[i]]['PROVINCIA'].values[idx:idx+1][0]
            name_l.append(name)
    #print(reg[i])
    flow_pro[reg[i]].insert(1, "NAME", name_l)
    #print(flow_pro[reg[i]]) 


In [406]:
#Display the results
import plotly.graph_objs as go
import plotly.figure_factory as ff

result=pd.concat(flow_pro.values(), axis=1)

#fig = go.Figure()

# plot the data 
fig5 = go.Figure()
color_dict = {'Lombardia':'#254607' ,'Emilia Romagna':'#417a0c', 'Friuli Venezia Giulia':'#5fb311'}
color=[color_dict[k] for k in flow['REGION'].values]
for i in range(len(reg)):  
        fig5.add_trace(go.Bar(x = flow_pro[reg[i]]["NAME"], y = flow_pro[reg[i]]["FLOW"], name=reg[i], marker_color=color[i]))
        fig5.update_layout(barmode='relative')
       # fig5.add_trace(go.Bar(x = flow_pro[reg[i]]["NAME"], y = flow_pro[reg[i]]["FLOW"],name =reg[i]))
fig5.show()

table_color =  [[0, '#004d00'],[.5, '#ffffff'],[1, '#ffffff']]
    
for i in range(len(reg)):  
    fig = ff.create_table(flow_pro[reg[i]],colorscale=table_color)
    fig.update_layout(xaxis = {'domain': [.2, .7]}, yaxis1 = {'domain': [0, 1]})
    fig.show()
    

In [407]:
#STUDY OF THE FLUXES BASED ON THE DAY OF THE WEEK

#Function that returns a dataframe with all the info (data_day_od solo) for the selected provinces for each region
def data_selected_province(data_day_od,cod_pro_selected):
    data_province_selected=data_day_od[data_day_od['COD_PRO'].isin(cod_pro_selected)] 
    return data_province_selected

#Define a dictionary with a dataframe for the different regions, containg all the info of the selected provinces 
d_selected_province = {} 
for i in range(len(reg)):
    d_selected_province[reg[i]] = pd.DataFrame(data_selected_province(data_day_od,flow_pro[reg[i]]['COD_PRO'].values[:]))
    
#print(d_selected_province)

#Sum of the fluxes based on the day of the week and the province
def flow_days(data_province_selected, cod_pro_selected):
    flow_days=data_province_selected.groupby(['DOW', 'COD_PRO']).FLOW.sum().reset_index()
    #flow_sorted=flow_days['FLOW'].sort_values(ascending=False)
    #flow_selected=flow_sorted.loc[flow_sorted[:] >= 1e4]
    #flow_selected= flow_sorted.head(n=3)
    return flow_days

#Define a dictionary with a dataframe 
d_flow_days = {} 

for i in range(len(reg)):
    d_flow_days[reg[i]] = pd.DataFrame(flow_days(d_selected_province[reg[i]],flow_pro[reg[i]]['COD_PRO']))


#Printing the dataframe associating the name to each province code
for i in range(len(reg)):
    name_list=[]
    for cod in d_flow_days[reg[i]]['COD_PRO'].values[:]:
        if cod in d[reg[i]]['COD_PRO'].values[:]:
            where=np.argwhere(d[reg[i]]['COD_PRO'].values[:]==cod)
            idx=where[0][0] #torna poco questo 00
            name=d[reg[i]]['PROVINCIA'].values[idx:idx+1][0]
            name_list.append(name)  
    #print(reg[i])
    d_flow_days[reg[i]].insert(1, "NAME", name_list)
    #print(d_flow_days[reg[i]])



#sort values
#print(flow_days_lombardia.sort_values(by='FLOW',ascending=False))
#print(flow_days_lombardia.sort_values(by='COD_PRO',ascending=False))
print("\n")

#Analysis based on the days

days = ['Lunedì', 'Martedì', 'Mercoledì', 'Giovedì', 'Venerdì', 'Sabato', 'Domenica']

#Function to order the dataset based on the days of the week 
def days_order(dataset,days):
    dataset['DOW']=pd.Categorical(dataset['DOW'], categories=days, ordered=True)
    dataset_days=dataset.sort_values(['DOW'], ignore_index=True)
    return dataset_days

#Function to sort the data by provinces
def sort(dataset):
    dataset=dataset.sort_values(['COD_PRO', 'NAME'],ascending=False, ignore_index=True)
    #dataset['DOW']=pd.Categorical(dataset['DOW'], categories=days, ordered=True)
    #dataset_days=dataset.sort_values(['DOW'])
    return dataset

#Define a dictionary cointaing a dataframe for each regions 
#Different dataframes based on different type of data organization
flow_each_pro={}

for i in range(len(reg)):
    list_k=[0]
    flow_each_pro[reg[i]]=sort(d_flow_days[reg[i]])
    for k in range(1,len(flow_each_pro[reg[i]]['COD_PRO'])):
        if flow_each_pro[reg[i]]['COD_PRO'][k]!=flow_each_pro[reg[i]]['COD_PRO'][k-1]:
            list_k.append(k)
    list_k.append(len(flow_each_pro[reg[i]]['COD_PRO']))
    for d in range(len(list_k)-1): 
            flow_each_pro[reg[i],d]= pd.DataFrame(days_order(flow_each_pro[reg[i]][list_k[d]:list_k[d+1]],days))
    flow_each_pro[reg[i]]=pd.DataFrame(days_order(d_flow_days[reg[i]],days))
    
    
#print(flow_each_pro[reg[0]].sort_values(['DOW','FLOW']))
#Data for a region, sorted by days of the week and diveded by provinces 
#print(flow_each_pro) 
#Data for a single province, sorted by the days of the week
#print(flow_each_pro[reg[i],k]) 

#for i in range(len(reg)):
    #for k in range(len(flow_pro[reg[i]]['COD_PRO'])):
        #print("\n", reg[i])
        #print(flow_each_pro[reg[i],k]) 
        #print("\n")
    
#Funtion to sum the fluxes of a day for each region 
def flux_sum_day(flow_each_pro):
    flux_sum_pd=flow_each_pro.groupby(['DOW'])['FLOW'].sum().reset_index()
    return flux_sum_pd

#Dictionary 
flux_sum_perday={}

#Dataframe for each region with the sum of fluxes based on the day of the week
for i in range(len(reg)):
        flux_sum_perday[reg[i]]=flux_sum_day(flow_each_pro[reg[i]])
        #print("\n", reg[i])
        #print(flux_sum_perday[reg[i]]) 
        


In [408]:
##########################PLOT#########################
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import iplot


# plot the data of provinces during the day for one region
fig3 = go.Figure()
color_dict = {'Lombardia':'#254607' ,'Emilia Romagna':'#417a0c', 'Friuli Venezia Giulia':'#5fb311'}
color=[color_dict[k] for k in flow['REGION'].values]
for i in range(len(reg)):  
        fig3 = fig3.add_trace(go.Scatter(x = flux_sum_perday[reg[i]]["DOW"], y = flux_sum_perday[reg[i]]["FLOW"],name =reg[i], marker=dict(color=color[i],size=8,line=dict(color=color[i],width=8))))
fig3.show()
# marker=dict(color='LightSkyBlue',size=12,line=dict(color='MediumPurple',width=12)

In [409]:

# plot the data of provinces during the day for one region
fig2 = go.Figure()
color_dict = [[' #008000',' #00cc00','#4dff4d' ], ['#ffa366','#b34700','#ff6600' ], ['#99003d','#ff99c2', '#ff1a75']]
#color_dict = {'Lombardia':'#254607' ,'Emilia Romagna':'#417a0c', 'Friuli Venezia Giulia':'#5fb311'}

for i in range(len(reg)):  
    for k in range(len(flow_pro[reg[i]]['COD_PRO'])):
        fig2 = fig2.add_trace(go.Bar(x = flow_each_pro[reg[i],k]["DOW"], y = flow_each_pro[reg[i],k]["FLOW"], name = flow_each_pro[reg[i],k].iloc[0,1], marker_color=color_dict[i][2-k]))
        #fig2.add_trace(go.Scatter(x = flow_each_pro[reg[i],k]["DOW"], y = flow_each_pro[reg[i],k]["FLOW"],name = flow_each_pro[reg[i],k].iloc[0,1]))
fig2.show()

# plot the data 
fig5 = go.Figure()



In [410]:
#MESIIIII

#Define a dictionary with a dataframe for the different regions, containg all the info of the selected provinces 
#print(d_selected_province)

################### MONTH
#Sum of the fluxes based on the month and the region (all province)
def flow_month_reg(data_reg):
    flow_m_reg=data_reg.groupby(['MONTH']).FLOW.sum().reset_index()
    return flow_m_reg

#Define a dictionary with a dataframe 
d_flow_month_reg = {} 

for i in range(len(reg)):
    d_flow_month_reg[reg[i]] = pd.DataFrame(flow_month_reg(data_reg[reg[i]]))
    

#Printing the dataframe associating the name to each province code
for i in range(len(reg)):
    print(reg[i])
    print(d_flow_month_reg[reg[i]])


######################province

#######################SORT BY MONTH!!!#######################################

#Sum of the fluxes based on the month and the province
def flow_month_pro(data_province_selected, cod_pro_selected):
    flow_m=data_province_selected.groupby(['MONTH', 'COD_PRO']).FLOW.sum().reset_index()
    #remve 'COD_PROV for months only'
    return flow_m

#Define a dictionary with a dataframe 
d_flow_month_pro = {} 

for i in range(len(reg)):
    d_flow_month_pro[reg[i]] = pd.DataFrame(flow_month_pro(d_selected_province[reg[i]],flow_pro[reg[i]]['COD_PRO']))


#Printing the dataframe associating the name to each province code
for i in range(len(reg)):
    print(reg[i])
    print(d_flow_month_pro[reg[i]].sort_values(by='COD_PRO'))


Lombardia
      MONTH   FLOW
0    Aprile  54213
1  Febbraio  38260
2    Maggio  50104
3     Marzo  44238
Emilia Romagna
      MONTH   FLOW
0    Aprile  26713
1  Febbraio  18483
2    Maggio  23071
3     Marzo  21079
Friuli Venezia Giulia
      MONTH   FLOW
0    Aprile  22447
1  Febbraio  17205
2    Maggio  20830
3     Marzo  20448
Lombardia
       MONTH  COD_PRO   FLOW
0     Aprile     15.0  32720
3   Febbraio     15.0  23729
6     Maggio     15.0  30827
9      Marzo     15.0  27274
1     Aprile     17.0   5174
4   Febbraio     17.0   4569
7     Maggio     17.0   5666
10     Marzo     17.0   4623
2     Aprile     20.0   5397
5   Febbraio     20.0   3934
8     Maggio     20.0   4740
11     Marzo     20.0   4540
Emilia Romagna
       MONTH  COD_PRO   FLOW
0     Aprile     36.0   2396
3   Febbraio     36.0   1679
6     Maggio     36.0   1717
9      Marzo     36.0   2156
1     Aprile     37.0  11632
4   Febbraio     37.0   8171
7     Maggio     37.0  10986
10     Marzo     37.0   8930
2    